# 데이터 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re
from tqdm import tqdm
import pickle
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
FOLDERNAME = 'image_project5'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

# Change dariectory to current folder
%cd /content/drive/MyDrive/$FOLDERNAME

In [ ]:
hrefs = []
pagenums = 119
baseurl = 'https://terms.naver.com/list.naver?cid=46702&categoryId=46739'
for page in range(1, pagenums+1):
    url = baseurl + f'&page={page}'
    resp = requests.get(url, headers = headers)
    soup = BeautifulSoup(resp.content, 'lxml')
    links = soup.select('a.thumb_link')
    for link in links:
        href = re.search(r'href="([^"]+)"', str(link)).group(1)
        hrefs.append(href)
    time.sleep(0.5)

In [ ]:
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

In [ ]:
content_list = []
count = 0

for href in tqdm(hrefs):
    url = f'https://terms.naver.com/{href}'
    resp = requests.get(url, headers=headers)

    if (resp.status_code == 200) & ('categoryId=46739' in resp.url): # 그런 페이지가 존재한다면 진행
        html = resp.content
        soup = BeautifulSoup(html, 'lxml')

        content_dict = {}
        content_dict["id"] = doc_id

        # 텍스트 정보
        tmp_profile = soup.select('div.tmp_profile')
        if tmp_profile:
            for tr in tmp_profile[0].select('tr'):
                if '<br/>' in str(tr.select('td')[0]): # 만약 여러 카테고리의 정보가 있다면
                    beforethis = re.findall("<b>(.+?)</b>", str(tr.select('td')[0]))[0] # 굵은 글씨로 된 단어(카테고리명) 중 첫번째
                    content_dict[tr.select('th')[0].text] = re.findall(f"(.+?){beforethis}", tr.select('td')[0].text)[0] # .text 중에 그거 전까지만
                    for category, content in zip(re.findall("<b>(.+?)</b>", str(tr.select('td')[0])), re.findall("</b> (.+?)<", str(tr.select('td')[0]))):
                        content_dict[category] = content
                else:
                    content_dict[tr.select('th')[0].text] = tr.select('td')[0].text

            content_list.append(content_dict)

            # 이미지
            img_url = soup.select('div.img_area img')[0]['src']
            img_resp = requests.get(img_url)
            with open(f'/content/drive/MyDrive/image_project5/data_naver_image/img_{doc_id}.png', 'wb') as f:
                f.write(img_resp.content)

    else: # 그런 페이지가 존재하지 않는다면
        continue

    time.sleep(0.5)

    if len(content_list) == 100:
        with open(f"/content/drive/MyDrive/image_project5/content_{count}00", "wb") as fw:
            pickle.dump(content_list, fw)
        count += 1
        content_list = []

In [ ]:
content_list_cplt = []
for i in range(0, 20):
    with open(f'/content/drive/MyDrive/image_project5/content_{i}00', 'rb') as f:
        content_list_cplt += pickle.load(f)

## 추가 데이터 크롤링

In [ ]:
def linkscrawling(artist):
    baseurl, pagenums = artistdict[artist]
    hrefs = []
    for page in range(1, pagenums+1):
        url = baseurl + f'&page={page}'
        resp = requests.get(url, headers = headers)
        soup = BeautifulSoup(resp.content, 'lxml')
        links = soup.select('a.thumb_link')
        for link in links:
            href = re.search(r'href="([^"]+)"', str(link)).group(1)
            hrefs.append(href)
        time.sleep(0.5)
    return hrefs

artistdict = {'강연균' : ['https://terms.naver.com/artsSearch.naver?query=%EA%B0%95%EC%97%B0%EA%B7%A0+%EB%AF%B8%EC%88%A0%EC%9E%91%ED%92%88', 5],
           '이해민선' : ['https://terms.naver.com/artsSearch.naver?query=%EC%9D%B4%ED%95%B4%EB%AF%BC%EC%84%A0+%EB%AF%B8%EC%88%A0%EC%9E%91%ED%92%88', 1],
           '박수근' : ['https://terms.naver.com/artsSearch.naver?query=%EB%B0%95%EC%88%98%EA%B7%BC', 3],
           '김환기' : ['https://terms.naver.com/artsSearch.naver?query=%EA%B9%80%ED%99%98%EA%B8%B0', 2],
           '김명식' : ['https://terms.naver.com/artsSearch.naver?query=%EA%B9%80%EB%AA%85%EC%8B%9D+%EB%AF%B8%EC%88%A0%EC%9E%91%ED%92%88', 6],
           '박영인' : ['https://terms.naver.com/artsSearch.naver?query=%EB%B0%95%EC%98%81%EC%9D%B8+%EB%AF%B8%EC%88%A0%EC%9E%91%ED%92%88', 4],
            '유영국' : ['https://terms.naver.com/artsSearch.naver?query=%EC%9C%A0%EC%98%81%EA%B5%AD', 2],
            '황술조' : ['https://terms.naver.com/artsSearch.naver?query=%ED%99%A9%EC%88%A0%EC%A1%B0', 1],
            '이인성' : ['https://terms.naver.com/artsSearch.naver?query=%EC%9D%B4%EC%9D%B8%EC%84%B1', 1]}

hrefs = []
for artist in artistdict.keys() :
    hrefs = hrefs + linkscrawling(artist)

#이하 위와 동일하게 데이터 수집

# 작가 분류

## 데이터 전처리

In [ ]:
!pip install tensorflow==2.9.0

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
!pip install split-folders

In [ ]:
!pip install datasets

In [ ]:
import splitfolders
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [ ]:
with open('/content/drive/MyDrive/image_project5/clean_content_list.pickle', 'rb') as f:
    content_list = pickle.load(f)

In [ ]:
a = content_list['아티스트'].value_counts()[content_list['아티스트'].value_counts() > 100]
content_list_artist = content_list[content_list['아티스트'].isin(a.index)]

In [ ]:
content_list_artist = content_list_artist.filter(items=['id', '아티스트'])
content_list_artist

In [ ]:
img_filedir = "/content/drive/MyDrive/image_project5/data_naver_image"
content_list_artist['filepath'] = content_list_artist['id'].apply(lambda x : img_filedir + '/img_' + x + '.png')
content_list_artist

In [ ]:
widths_temp = []
lengths_temp = []
for img_path in content_list_artist['filepath']:
    img = cv2.imread(img_path)
    widths_temp.append(img.shape[0])
    lengths_temp.append(img.shape[1])

In [ ]:
widths_temp, lengths_temp = np.array(widths_temp), np.array(lengths_temp)

In [ ]:
count = 0
for w, l in zip(widths_temp, lengths_temp) :
    if (l * 2.42 < w) or (w * 2.42 < l) :
        count += 1
count

In [ ]:
display(np.mean(widths_temp), np.mean(lengths_temp))
display(np.max(widths_temp), np.max(lengths_temp))
display(np.min(widths_temp), np.min(lengths_temp))

In [ ]:
thres = 256
content_list_artist.iloc[np.concatenate((np.where(widths_temp < thres)[0], np.where(lengths_temp < thres)[0]))]

In [ ]:
content_list_artist = content_list_artist.drop(index=content_list_artist.iloc[np.concatenate((np.where(widths_temp < thres)[0], np.where(lengths_temp < thres)[0]))].index)

In [ ]:
import imutils

In [ ]:
img_size = 224

In [ ]:
def rotate_and_resize_and_pad(img):
    # ROTATE
    if (img.shape[1] * 2.42 < img.shape[0]) or (img.shape[0] * 2.42 < img.shape[1]) :
        img = imutils.rotate_bound(img, 45)

    # RESIZE
    if(img.shape[1] > img.shape[0]) :
        ratio = img_size/img.shape[1]
    else :
        ratio = img_size/img.shape[0]
    img = cv2.resize(img, dsize=(0, 0), fx=ratio, fy=ratio, interpolation=cv2.INTER_AREA) # INTER_LANCZOS4

    # PAD
    w, h = img.shape[1], img.shape[0]
    dw, dh = (img_size-w)/2, (img_size-h)/2
    M = np.float32([[1,0,dw], [0,1,dh]])
    img_padded = cv2.warpAffine(img, M, (img_size, img_size))

    return img_padded

In [ ]:
label_names = content_list_artist['아티스트'].unique()

In [ ]:
label_names

In [ ]:
label2index = {}
for i, name in enumerate(label_names):
    label2index[name] = i
label2index

In [ ]:
os.mkdir('/content/resize')

In [ ]:
for label in label_names:
    dir_path = '/content/resize/'+ str(label2index[label])
    os.mkdir(dir_path)

In [ ]:
for img_path, label in tqdm(zip(content_list_artist['filepath'], content_list_artist['아티스트']), total=content_list_artist.shape[0]):
    img = cv2.imread(img_path)
    img_padded = rotate_and_resize_and_pad(img)
    filename = img_path.split('/')[-1]
    cv2.imwrite(f'/content/resize/{label2index[label]}/{filename}', img_padded)

In [ ]:
splitfolders.ratio('/content/resize', output='dataset_224_rotate', seed=77, ratio=(0.8, 0.1, 0.1))

In [ ]:
folder_path = "/content/drive/MyDrive/image_project5/dataset_224_rotate"
for t in ['train', 'val', 'test']:
    for label in label_names:
        print(t, label, len(os.listdir(folder_path + '/' + t + '/' + str(label2index[label]))))

In [ ]:
# 이미지 데이터가 있는 폴더 경로 설정
dataset_name = 'dataset_224_rotate'

train_folder_path = "/content/drive/MyDrive/image_project5/" + dataset_name + "/train"
val_folder_path = "/content/drive/MyDrive/image_project5/" + dataset_name + "/val"
test_folder_path = "/content/drive/MyDrive/image_project5/" + dataset_name + "/test"

### mean img for zero-centering

In [ ]:
train_dirdict = {}
for label in label_names:
    filenames = os.listdir(train_folder_path + "/" + str(label2index[label]))
    filenames = list(map(lambda filename : train_folder_path + "/" + str(label2index[label]) + "/" + filename, filenames))
    train_dirdict[label2index[label]] = filenames

In [ ]:
x_train = []
for label in label_names:
    for img_path in train_dirdict[label2index[label]]:
        img = cv2.imread(img_path)
        x_train.append(img)

In [ ]:
len(x_train)

In [ ]:
compute_mean = lambda imgs : np.mean(imgs, axis=0)

In [ ]:
mean_img = compute_mean(x_train)

In [ ]:
mean_img.shape

In [ ]:
with open("/content/drive/MyDrive/image_project5/mean_img_artist10_224_rotate.pickle", "wb") as fw:
    pickle.dump(mean_img, fw)

In [ ]:
x_train = np.array(x_train)

In [ ]:
x_train_gray = tf.image.rgb_to_grayscale(x_train)

In [ ]:
x_train_gray.shape

In [ ]:
mean_img_gray = compute_mean(x_train_gray)

In [ ]:
mean_img_gray.shape

In [ ]:
with open("/content/drive/MyDrive/image_project5/mean_img_artist10_224_rotate_gray.pickle", "wb") as fw:
    pickle.dump(mean_img_gray, fw)

In [ ]:
# mean img 불러오기(rgb와 grayscale 2개)
with open('/content/drive/MyDrive/image_project5/mean_img_artist10_224_rotate.pickle', 'rb') as f:
    mean_img = pickle.load(f)
with open('/content/drive/MyDrive/image_project5/mean_img_artist10_224_rotate_gray.pickle', 'rb') as f:
    mean_img_gray = pickle.load(f)

In [ ]:
batch_size = 128

In [ ]:
train_dataset = image_dataset_from_directory(train_folder_path,
                                            batch_size=batch_size,
                                            image_size=(img_size, img_size),
                                            shuffle=True,
                                            seed=39,
                                            interpolation='lanczos3')

val_dataset = image_dataset_from_directory(val_folder_path,
                                            batch_size=batch_size,
                                            image_size=(img_size, img_size),
                                            shuffle=True,
                                            seed=39,
                                            interpolation='lanczos3')

test_dataset = image_dataset_from_directory(test_folder_path,
                                            batch_size=batch_size,
                                            image_size=(img_size, img_size),
                                            shuffle=True,
                                            seed=39,
                                            interpolation='lanczos3')

## 모델링

In [ ]:
num_classes = 10

### Model Base

In [ ]:
enb4 = tf.keras.applications.EfficientNetB4(include_top=False,
                                            weights="imagenet",
                                            input_shape=None,
                                            pooling=None)

In [ ]:
enb4.trainable = False

In [ ]:
inputs = tf.keras.Input(shape=(img_size, img_size, 3,))
x = sub_mean_img(inputs, mean_img)
x = enb4(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
# callbacks
es = early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(train_dataset, batch_size=batch_size, validation_data=val_dataset, epochs=30,
                    callbacks=[es])

### Model v2 (380)

In [ ]:
img_size = 380

In [ ]:
inputs = tf.keras.Input(shape=(img_size, img_size, 3,))
x = sub_mean_img(inputs, mean_img)
x = enb4(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
# callbacks
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
ck = tf.keras.callbacks.ModelCheckpoint(filepath, verbose=1, save_best_only=True)

In [ ]:
history = model.fit(train_dataset, batch_size=batch_size, validation_data=val_dataset, epochs=30,
                    callbacks=[es, ck])

### Model v3 (grayscale)

In [ ]:
enb0_1 = tf.keras.applications.EfficientNetB0(include_top=False,
                                            weights="imagenet",
                                            input_shape=None,
                                            pooling=None)

enb0_2 = tf.keras.applications.EfficientNetB0(include_top=False,
                                            weights="imagenet",
                                            input_shape=None,
                                            pooling=None)

In [ ]:
# 분류층만 학습시
enb0_1.trainable = False
enb0_2.trainable = False

In [ ]:
enb0_1._name = "efficientnetb0_1"
enb0_2._name = "efficientnetb0_2"

In [ ]:
inputs = tf.keras.Input(shape=(img_size, img_size, 3,))

x_rgb = sub_mean_img(inputs, mean_img)
x_rgb = enb0_1(x_rgb, training=False)
x_rgb = tf.keras.layers.GlobalAveragePooling2D()(x_rgb)

x_gray = tf.image.rgb_to_grayscale(inputs)
x_gray = sub_mean_img(x_gray, mean_img_gray)
x_gray = tf.repeat(x_gray, 3, axis=3)
x_gray = enb0_2(x_gray, training=False)
x_gray = tf.keras.layers.GlobalAveragePooling2D()(x_gray)

x = tf.keras.layers.Concatenate(axis=-1)([x_rgb, x_gray])

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# tf.keras.optimizers.AdamW(learning_rate=0.0001)

In [ ]:
# callbacks
ck_path = "/content/drive/MyDrive/image_project5/model_224_gray_onlytop"

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
ck = tf.keras.callbacks.ModelCheckpoint(ck_path, verbose=1, save_best_only=True)

In [ ]:
history = model.fit(train_dataset, batch_size=batch_size, validation_data=val_dataset, epochs=30,
                    callbacks=[es])

### Model v3-1 (rotate)

In [ ]:
# 재사용할 레이어
dense = tf.keras.layers.Dense(256, activation='relu')

# 연결
inputs = tf.keras.Input(shape=(img_size, img_size, 3,))

x_rgb = sub_mean_img(inputs, mean_img)
x_rgb = enb0_1(x_rgb, training=False)
x_rgb = tf.keras.layers.GlobalAveragePooling2D()(x_rgb)

x_gray = tf.image.rgb_to_grayscale(inputs)
x_gray = sub_mean_img(x_gray, mean_img_gray)
x_gray = tf.repeat(x_gray, 3, axis=3)
x_gray = enb0_2(x_gray, training=False)
x_gray = tf.keras.layers.GlobalAveragePooling2D()(x_gray)

x = tf.keras.layers.Concatenate(axis=-1)([x_rgb, x_gray]) # (, 2560)

x = dense(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# tf.keras.optimizers.AdamW(learning_rate=0.0001)

In [ ]:
ck_path = "/content/drive/MyDrive/image_project5/model_224_gray_rotate_onlytop"

In [ ]:
# callbacks
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
ck = tf.keras.callbacks.ModelCheckpoint(ck_path, verbose=1, save_best_only=True)

In [ ]:
history = model.fit(train_dataset, batch_size=batch_size, validation_data=val_dataset, epochs=30,
                    callbacks=[es, ck])

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.save('onlytop_224_gray_rotate.h5')

#### weight 분석

In [ ]:
# 가중치가 궁금해!
model.layers[-3].weights[0]

In [ ]:
rgb_weights = model.layers[-3].weights[0][:1280]
gray_weights = model.layers[-3].weights[0][1280:]

In [ ]:
rgb_weights_np = np.array(rgb_weights)
gray_weights_np = np.array(gray_weights)

In [ ]:
rgb_weights_abs.sum(), gray_weights_abs.sum()

In [ ]:
np.mean(rgb_weights_abs), np.mean(gray_weights_abs)

### Model v4 (augmentation)

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomBrightness(0.2, seed=39),
        tf.keras.layers.RandomContrast(0.8, seed=39),
    ]
)

In [ ]:
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))

In [ ]:
# 재사용할 레이어
dense = tf.keras.layers.Dense(256, activation='relu')

# 연결
inputs = tf.keras.Input(shape=(img_size, img_size, 3,))

x_rgb = sub_mean_img(inputs, mean_img)
x_rgb = enb0_1(x_rgb, training=False)
x_rgb = tf.keras.layers.GlobalAveragePooling2D()(x_rgb)

x_gray = tf.image.rgb_to_grayscale(inputs)
x_gray = sub_mean_img(x_gray, mean_img_gray)
x_gray = tf.repeat(x_gray, 3, axis=3)
x_gray = enb0_2(x_gray, training=False)
x_gray = tf.keras.layers.GlobalAveragePooling2D()(x_gray)

x = tf.keras.layers.Concatenate(axis=-1)([x_rgb, x_gray])

x = dense(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
# callbacks
ck_path = "/content/drive/MyDrive/image_project5/model_224_gray_rotate_aug_onlytop"

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
ck = tf.keras.callbacks.ModelCheckpoint(ck_path, verbose=1, save_best_only=True)

In [ ]:
history = model.fit(train_dataset, batch_size=batch_size, validation_data=val_dataset, epochs=30,
                    callbacks=[es, ck])

In [ ]:
model.save('model_224_gray_rotate_aug_onlytop.h5')

In [ ]:
model.evaluate(test_dataset)

### Model v5 (dropout 0.7)

In [ ]:
# 재사용할 레이어
dense = tf.keras.layers.Dense(512, activation='relu')

# 연결
inputs = tf.keras.Input(shape=(img_size, img_size, 3,))

x_rgb = sub_mean_img(inputs, mean_img)
x_rgb = enb0_1(x_rgb, training=False)
x_rgb = tf.keras.layers.GlobalAveragePooling2D()(x_rgb)

x_gray = tf.image.rgb_to_grayscale(inputs)
x_gray = sub_mean_img(x_gray, mean_img_gray)
x_gray = tf.repeat(x_gray, 3, axis=3)
x_gray = enb0_2(x_gray, training=False)
x_gray = tf.keras.layers.GlobalAveragePooling2D()(x_gray)

x = tf.keras.layers.Concatenate(axis=-1)([x_rgb, x_gray]) # (, 2560)

x = dense(x)
x = tf.keras.layers.Dropout(0.7)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
# callbacks
ck_path = "/content/drive/MyDrive/image_project5/model_224_gray_rotate_dropout07_onlytop"

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
ck = tf.keras.callbacks.ModelCheckpoint(ck_path, verbose=1, save_best_only=True)

In [ ]:
history = model.fit(train_dataset, batch_size=batch_size, validation_data=val_dataset, epochs=30,
                    callbacks=[es, ck])

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.save('model_224_gray_rotate_dropout07_onlytop.h5')

# 기법 분류

## EDA & 라벨 선정

In [ ]:
with open('/content/drive/MyDrive/4조 이미지/plus_content.pickle', 'rb') as f:
    data = pickle.load(f)

data = pd.DataFrame(data)
data

In [ ]:
data['기법'].value_counts()

In [ ]:
data.isnull().sum()

In [ ]:
def remove_non_korean(word):
    if pd.isna(word):
        return word
    else:
        return re.sub('[^가-힣]', '', word)
data['기법'] = data['기법'].apply(remove_non_korean)
data['아티스트'] = data['아티스트'].apply(remove_non_korean)

In [ ]:
def replaceword(preword, result):
    try:
        if result in preword :
            return result
        else :
            return preword
    except TypeError:
        return preword

data['기법'] = data['기법'].apply(lambda x: replaceword(x, '수채'))
data['기법'] = data['기법'].apply(lambda x: replaceword(x, '유채'))

combineword = {'종이에수묵' : '지본수묵',
               '종이에수묵담채' : '지본담채',
               '종이에수묵채색' : '지본채색',
               '비단에채색' : '견본채색',
               '비단에수묵담채' : '견본담채',
               '비단에수묵' : '견본수묵',
               '종이에먹' : '지본수묵',
               '종이에담채' : '지본담채',
               '지본수묵담채' : '지본담채',
               '비단에수묵채색' : '견본채색',
               '한지위에채색' : '한지에수묵채색',
               '비단에먹' : '견본수묵',
               '종이에채색' : '지본채색',
               '지본수묵채색' : '지본채색'}

data['기법'].replace(combineword, inplace = True)

## 데이터 전처리

In [ ]:
img_zip = zipfile.ZipFile('/content/drive/MyDrive/4조 이미지/filtered_plus_image.zip')
img_zip.extractall(path = '/content/filtered_plus_image')
folder_path = '/content/filtered_plus_image'
listdir = os.listdir(folder_path)
id = [x[4:-4] for x in listdir]
imgdict = {'id' : id,
           'img_file' : listdir}
data.drop('img_file',axis = 1, inplace = True)
imgdf = pd.DataFrame(imgdict)
data2= pd.merge(data, imgdf, how = 'outer')

In [ ]:
data2

In [ ]:
data2.dropna(subset = ['img_file'], inplace = True)
color_data = data2[data2['기법'].isin(['지본채색', '지본담채', '지본수묵', '수채', '유채'])]
color_data = color_data[['id','아티스트','기법', 'img_file']]

In [ ]:
color_data['기법'].value_counts()

In [ ]:
labels = ['지본채색', '지본담채', '지본수묵', '수채', '유채']
majorlabel1 = color_data[color_data['기법'] == '지본수묵']
majorlabel2 = color_data[color_data['기법'] == '지본담채']

down_majorlabel1 = resample(majorlabel1, replace=False, n_samples=513, random_state=42)
down_majorlabel2 = resample(majorlabel2, replace=False, n_samples=513, random_state=42)

balanced_color_data = pd.concat([down_majorlabel1, down_majorlabel2,color_data[color_data['기법'].isin(['지본채색', '유채', '수채'])]])

df = balanced_color_data

###Resizing, padding

In [ ]:
#한 장의 이미지의 경로를 받아 resizing, padding하는 함수

def resize(img_path, INTERPOL = cv2.INTER_AREA , img_size = 224):
    img = cv2.imread(img_path)

    #rotate
    if (img.shape[1] * 2.42 < img.shape[0]) or (img.shape[0] * 2.42 < img.shape[1])   :
        img = imutils.rotate_bound(img, 45)

    #resize
    if(img.shape[1] > img.shape[0]) :
        ratio = img_size/img.shape[1]
    else :
        ratio = img_size/img.shape[0]
    img = cv2.resize(img, dsize = (0,0), fx=ratio, fy=ratio, interpolation=INTERPOL)

    #검은색으로 칠하기
    w, h = img.shape[1], img.shape[0]

    dw = (img_size-w)/2 # img_size와 w의 차이
    dh = (img_size-h)/2 # img_size와 h의 차이

    M = np.float32([[1,0,dw], [0,1,dh]])  #(2*3 이차원 행렬)
    img = cv2.warpAffine(img, M, (img_size, img_size)) #이동변환

    return img

images, labels = [], []
for idx in tqdm(range(len(df))):
    id = df['id'].iloc[idx]
    label = df['기법'].iloc[idx]
    img_path = '/content//filtered_plus_image' + '/' + f'img_{id}.png'
    images.append(resize(img_path, INTERPOL = cv2.INTER_LANCZOS4))
    labels.append(label)

### 라벨 인코딩, 이미지 생성, zero-centering

In [ ]:
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(label_encoded)

images_adj = tf.image.adjust_contrast(images, contrast_factor = 2.)

#list라면 array 변환
if isinstance(images, np.ndarray) == False:
    images = np.array(images)
    labels = np.array(labels)

#split
x_data, x_val, y_data, y_val = train_test_split(images, labels,
                                                    test_size=0.1,
                                                    shuffle=True,
                                                    random_state=42)

#split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=42)

np.save('/content/drive/MyDrive/backup/npy/x_train.npy', x_train)
np.save('/content/drive/MyDrive/backup/npy/x_test.npy', x_test)
np.save('/content/drive/MyDrive/backup/npy/x_val.npy', x_val)
np.save('/content/drive/MyDrive/backup/npy/y_train.npy', y_train)
np.save('/content/drive/MyDrive/backup/npy/y_test.npy', y_test)
np.save('/content/drive/MyDrive/backup/npy/y_val.npy', y_val)

In [ ]:
# CONTRAST
x_train_adj = tf.image.adjust_contrast(x_train, contrast_factor = 1.5)
x_test_adj = tf.image.adjust_contrast(x_test, contrast_factor = 1.5)

np.save('/content/drive/MyDrive/backup/npy/x_train_adj.npy', x_train_adj)
np.save('/content/drive/MyDrive/backup/npy/x_test_adj.npy', x_test_adj)

compute_mean = lambda imgs : np.mean(imgs, axis=0).astype('int8')
mean_img = compute_mean(x_train)
mean_img_adj = compute_mean(x_train_adj)
def sub_mean_img(imgs, mean_img):
    return imgs - mean_img

In [ ]:
# GRAYSCALE
images_gray = []
for image in images :
    img_gray = rgb2gray(image)
    img_gray = np.repeat(img_gray[:, :, np.newaxis], 3, axis = 2)
    images_gray.append(img_gray)
len(images_gray)

## 모델링

In [ ]:
# MAIN : concat contrast-adjusted image
num_classes = 5
image_size = 224
batch_size = 32

base_model1 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3))
base_model2 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3))
Embedding_Layer = Dense(256, activation = 'relu', name = 'Embedding_Layer')

base_model1._name = 'efficientnetb0_1'
base_model2._name = 'efficientnetb0_2'

input1 = tf.keras.Input(shape = (image_size, image_size, 3))
input2 = tf.keras.Input(shape = (image_size, image_size, 3))

x1 = sub_mean_img(input1, mean_img)
x1 = base_model1(x1)
x1 = GlobalAveragePooling2D()(x1)

x2 = sub_mean_img(input2, mean_img_adj)
x2 = base_model2(x2)
x2 = GlobalAveragePooling2D()(x2)

concat = tf.keras.layers.Concatenate(axis=-1)([x1,x2])
concat = Dense(512, activation='relu')(concat)
concat = Dropout(0.5)(concat)
embedding = Embedding_Layer(concat)
embedding = Dropout(0.5)(embedding)
predictions = Dense(num_classes, activation='softmax')(embedding)

model = Model(inputs=[input1, input2], outputs=predictions)


# Freeze
for layer in base_model1.layers:
    layer.trainable = False

for layer in base_model2.layers:
    layer.trainable = False

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
ck_path = "/content/drive/MyDrive/backup/model"
ck = tf.keras.callbacks.ModelCheckpoint(ck_path, verbose=1, save_best_only=True)

# Train
history = model.fit(
    [x_train, x_train_adj],
    y_train,
    batch_size = batch_size,
    epochs = 100,
    validation_data = ([x_test, x_test_adj], y_test),
    steps_per_epoch=len(x_train) // batch_size,
    validation_steps=len(x_test) // batch_size,
    callbacks=[early_stopping, ck]
)

model.save('/content/drive/MyDrive/backup/style_contrast_2depth.h5')

### Grayscale concat 코드

In [ ]:
x_train_gray, x_test_gray, y_train, y_test = split2center(images_gray, labels)

num_classes = len(label_encoder.classes_)
image_size = 224
batch_size = 32


base_model1 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3))
base_model2 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3))

base_model1._name = 'model1'
base_model2._name = 'model2'

input1 = tf.keras.Input(shape = (image_size, image_size, 3))
input2 = tf.keras.Input(shape = (image_size, image_size, 3))

x1 = base_model1(input1)
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model2(input2)
x2 = GlobalAveragePooling2D()(x2)

concat = tf.keras.layers.Concatenate(axis=-1)([x1,x2])
concat = Dense(256, activation='relu')(concat)
concat = Dropout(0.3)(concat)
predictions = Dense(num_classes, activation='softmax')(concat)

model2 = Model(inputs=[input1, input2], outputs=predictions)

# Freeze
for layer in base_model1.layers:
    layer.trainable = False

# Freeze
for layer in base_model2.layers:
    layer.trainable = False


# Compile the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with early stopping
history = model2.fit(
    [x_train, x_train_gray],
    y_train,
    batch_size = batch_size,
    epochs = 100,
    validation_data = ([x_test, x_test_gray], y_test),
    steps_per_epoch=len(x_train) // batch_size,
    validation_steps=len(x_test) // batch_size,
    callbacks=[early_stopping]
)

###Freeze 10% 해제 코드

In [ ]:
tf.keras.backend.clear_session()
# Freeze 10% 해제 코드

num_classes = 5
image_size = 224
batch_size = 32

base_model1 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3))
base_model2 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3))
Embedding_Layer = Dense(256, activation = 'relu', name = 'Embedding_Layer')

base_model1._name = 'efficientnetb0_1'
base_model2._name = 'efficientnetb0_2'

input1 = tf.keras.Input(shape = (image_size, image_size, 3))
input2 = tf.keras.Input(shape = (image_size, image_size, 3))

x1 = sub_mean_img(input1, mean_img)
x1 = base_model1(x1)
x1 = GlobalAveragePooling2D()(x1)

x2 = sub_mean_img(input2, mean_img_adj)
x2 = base_model2(x2)
x2 = GlobalAveragePooling2D()(x2)

concat = tf.keras.layers.Concatenate(axis=-1)([x1,x2])
concat = Dense(512, activation='relu')(concat)
concat = Dropout(0.3)(concat)
embedding = Embedding_Layer(concat)
embedding = Dropout(0.2)(embedding)
predictions = Dense(num_classes, activation='softmax')(embedding)

model = Model(inputs=[input1, input2], outputs=predictions)


# Freeze
for layer in base_model1.layers[:-23]:
    layer.trainable = False

# Freeze
for layer in base_model2.layers[:-23]:
    layer.trainable = False

for layer in base_model1.layers[-23:]:
    layer.trainable = True

# Freeze
for layer in base_model2.layers[-23:]:
    layer.trainable = True


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
ck_path = "/content/drive/MyDrive/backup/model"
ck = tf.keras.callbacks.ModelCheckpoint(ck_path, verbose=1, save_best_only=True)
# Train the model with early stopping
history = model.fit(
    [x_train, x_train_adj],
    y_train,
    batch_size = batch_size,
    epochs = 100,
    validation_data = ([x_test, x_test_adj], y_test),
    steps_per_epoch=len(x_train) // batch_size,
    validation_steps=len(x_test) // batch_size,
    callbacks=[early_stopping, ck]
)

model.save('/content/drive/MyDrive/backup/melt_style_contrast_2depth.h5')

### Evaluate

In [ ]:
x_val_adj = tf.image.adjust_contrast(x_val, contrast_factor = 1.5)
model.evaluate([x_val,x_val_adj], y_val)

###Similarity

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/backup/style_contrast_2depth.h5)

In [ ]:
embedding_model = Model(inputs = [model.input[0], model.input[1]],
                        outputs = model.get_layer('Embedding_Layer').output)

embeddings = embedding_model.predict([x_train, x_train_adj])
similarity_matrix = cosine_similarity(embeddings)

In [ ]:
def recommend(query, num = 5, images = x_train):
    # Recommend similar images
    similar_images_indices = np.argsort(similarity_matrix[query])[::-1][1:num+1]  # Top 5 similar images

    rows = int(np.sqrt(num))
    cols = int(np.ceil(num / rows))
    fig, axes = plt.subplots(rows, cols, figsize=(10, 10))

    similar_images = []
    for index in similar_images_indices:
        similar_images.append(images[index])

    similar_images = [images[query]] + similar_images

    print("Similar Images:")
    for i, image in enumerate(similar_images):
        ax = axes[(i) // cols, (i) % cols]  # Get the current subplot
        ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Plot the image
        #ax.axis('off')

    plt.tight_layout()

    plt.show()

In [ ]:
recommend(256)

In [ ]:
recommend(3)